In [5]:
# imports
from pkg_resources import resource_filename
from matplotlib import pyplot as plt
import numpy as np

from frb.galaxies import frbgalaxy
from frb import frb

from astropy.coordinates import SkyCoord
from astropy import units
from astropy.cosmology import Planck15

from frb.galaxies import defs
from frb import utils

## Test for one FRB

In [9]:
frb190611 = frb.FRB.by_name('FRB190611')
HG190611 = frb190611.grab_host()

## Define points within FRB localization

In [14]:
a = np.sqrt(frb190611.eellipse['a_sys'] ** 2 + frb190611.eellipse['a'] ** 2)
b = np.sqrt(frb190611.eellipse['b_sys'] ** 2 + frb190611.eellipse['b'] ** 2)
in_local = (frb190611.coord.separation(frb190611.coord) < a * units.arcsec) | (frb190611.coord.separation(frb190611.coord) < b * units.arcsec)

## Make 2D gaussian distribution from a and b

In [19]:
def gauss_2d(centroid, offset):
    x = random.gauss(frb190611.coord, a)
    y = random.gauss(frb190611.coord, b)
    return (x, y)

In [20]:
plt.plot(x,y)

NameError: name 'x' is not defined